In [21]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline


In [22]:
#Importar data
FR_df=pd.read_csv("FRvideos_cc50_202101.csv")

In [23]:
#Importar el json y extraer la data que nos interesa
import json
FR_category={}
with open("FR_category_id.json") as f:
    FR_data=json.load(f)
for i in list(pd.DataFrame(FR_data['items']).id.values):
    FR_category[i]=pd.DataFrame(FR_data['items'])[pd.DataFrame(FR_data['items']).id==i].snippet.values[0]['title']

In [24]:
#vemos la informacion de toda la informacion de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [29]:
FR_df.sample(3)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,state,lat,lon,geometry
12044,HHx0txNcTGU,18.15.01,Sérieux papa? seulement 20 000$ pour mon anniv?,CoD Forlan,24,2018-01-14T19:16:10.000Z,"top 10""|""top 5""|""top 10 gosse de riche""|""top 5 gosse de riche""|""top 10 enfant riche""|""top 5 enfant riche""|""enfant riche""|""enfant pourri gaté""|""gosse de riche""|""enfant de riche""|""spoiled kids""|""argent""|""riche vs pauvre""|""voiture de luxe""|""luxe""|""snapchat""|""snap""|""les pire snap de riche""|""les pires snap de riche""|""les pires snapchat""|""les pires snapchat de riche""|""les pires snapchat de personnes riches""|""les pire snapchat de riche""|""pere riche""|""compilation""|""reaction",28687,3389,356,1688,https://i.ytimg.com/vi/HHx0txNcTGU/default.jpg,False,False,False,"Ce gosse de riche m'a fait mal au coeur...Certains enfants ont de la chance d'avoir un père très riche qui leur offre tout ce qu'ils veulent...Mais le problème, c'est qu'il se vantent un peu trop sur les réseaux sociaux comme par exemple sur Snap...N'oubliez pas que l'argent ne fait pas le bonheur ! \n\nOriginal Video : https://www.youtube.com/watch?v=iz3NTVfjUEA&t=11s\n\n🎮 ABONNE TOI ICI : https://www.youtube.com/user/Codforlan?sub_confirmation=1\n\n🎮 LIKE & PARTAGE CETTE VIDÉO POUR DEVENIR UN MAFIEUX ! \nDÉROULE CETTE DESCRIPTION !!!\n\n🎮 TWITTER : https://twitter.com/CoDforlan24\n🎮 FACEBOOK : https://fr-fr.facebook.com/cod.forlan\n🎮 TWITCH : https://www.twitch.tv/forlan_youtuber\n🎮 SNAPCHAT : Codforlan24\n\n------------------------------------------------INFOS-------------------------------------------------\n\nAujourd'hui les amis on se retrouve pour une vidéo assez sympathique et agréable à regarder : L'histoire de l'anniversaire d'un gosse de riche qui va mal tourner ! \n\nJe vous invite à LIKER, COMMENTER & PARTAGER la vidéo si tu veux devenir un Mafieux fidèle !\n\n------------------DEVENIR UN MEMBRE DE LA MAFIA SQUAD------------------\n\n🎮 Si tu veux devenir un VRAI membre de la Mafia Squad, mets notre logo sur tes comptes Twitter, Youtube et Twitch le lien du téléchargement se trouve ici :\n\nLogo en Rouge : http://www.mediafire.com/file/4t51o7nuch3zor5/MAFIA-SQUAD-rouge-v1.jpg\n\nLogo en Bleu : http://www.mediafire.com/download/7udbn187jmd42w6/MAFIA-SQUAD_bleu.jpg\n\nLogo en Jaune : http://www.mediafire.com/download/212x0osndg67pi7/MAFIA-SQUAD_jaune.jpg\n\nLogo en Orange : http://www.mediafire.com/download/9tdnvbehsg1b5xr/MAFIA-SQUAD_orange.jpg\n\nLogo en Rose : http://www.mediafire.com/download/1ulitzqxvv8t8ga/MAFIA-SQUAD_rose.jpg\n\nLogo en Vert : http://www.mediafire.com/download/9vkcngmbw2pa70m/MAFIA-SQUAD_vert.jpg\n\nLogo en Violet : http://www.mediafire.com/download/l0axm4s5j755i54/MAFIA-SQUAD_voilet.jpg\n\nLogo en Noir : http://www.mediafire.com/file/ewvzns88wc5khgn/%5E50F14C36479471DD5D31374B1D1FEB26642782324BDCB2FC5B%5Epimgpsh_fullsize_distr.jpg\n\nAllez portez vous bien les Mafieux, ciao tout le monde!\n\nMusique d'intro : G rex shut up :)",Moyotte,-12.787089,45.275004,POINT (45.27500362 -12.78708901)
39563,MfKQQ4yUFMg,18.08.06,CE MEC EST FOU ! REACTION COMPILATION SKATE,Mafiastunting,20,2018-06-07T19:14:24.000Z,CE MEC EST FOU ! REACTION COMPILATION SKATE,29170,2474,34,422,https://i.ytimg.com/vi/MfKQQ4yUFMg/default.jpg,False,False,False,CE MEC EST FOU ! REACTION COMPILATION SKATEStrictly skateboarding : https://www.youtube.com/watch?v=pMRv83QhLsMiFath SkateVids : https://www.youtube.com/watch?v=BawDOv8Iq3MMetro Skateboarding : https://www.youtube.com/watch?v=HFBC8Z3jcAc,Guadeloupe,16.241475,-61.532999,POINT (-61.5329989 16.24147504)
19059,JtepWmBpCi4,18.19.02,Le Zap de Cokaïn.fr n°039,Cokaïn.fr,24,2018-02-17T12:16:34.000Z,"Cokaïn.fr""|""spi0n""|""spi0n.com""|""spion""|""zap""|""zaping""|""drole""|""fail""|""compilation""|""cokain""|""cokain.fr""|""iZap4U""|""best""|""humour""|""fun""|

In [19]:
#Analizando  la media, moda,desv estandar, etc.
FR_df.describe()

,category_id,views,likes,dislikes,comment_count,lat,lon
count,40724.000000,4.072400e+04,4.072400e+04,4.072400e+04,4.072400e+04,40724.000000,40724.000000
mean,20.123809,4.199219e+05,1.738886e+04,8.149624e+02,1.832453e+03,36.986535,-2.314779
std,6.984422,1.772130e+06,8.720509e+04,1.139219e+04,1.404321e+04,19.889834,25.443453
min,1.000000,2.230000e+02,0.000000e+00,0.000000e+00,0.000000e+00,-21.033511,-61.532999
25%,17.000000,1.697450e+04,3.380000e+02,1.800000e+01,5.600000e+01,42.699989,-0.530030
50%,23.000000,7.372100e+04,1.892500e+03,8.300000e+01,2.350000e+02,45.899975,2.666648
75%,24.000000,2.708088e+05,7.969500e+03,3.350000e+02,8.410000e+02,48.516663,6.030009
max,44.000000,1.009116e+08,4.750254e+06,1.353661e+06,1.040912e+06,50.283325,55.712816


In [20]:
#Analizando los tipos de datos
FR_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40724 entries, 0 to 40723
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   video_id                40724 non-null  object 
 1   trending_date           40724 non-null  object 
 2   title                   40724 non-null  object 
 3   channel_title           40724 non-null  object 
 4   category_id             40724 non-null  int64  
 5   publish_time            40724 non-null  object 
 6   tags                    40724 non-null  object 
 7   views                   40724 non-null  int64  
 8   likes                   40724 non-null  int64  
 9   dislikes                40724 non-null  int64  
 10  comment_count           40724 non-null  int64  
 11  thumbnail_link          40724 non-null  object 
 12  comments_disabled       40724 non-null  bool   
 13  ratings_disabled        40724 non-null  bool   
 14  video_error_or_removed  40724 non-null

In [32]:
#Obtener la cantidad de filas duplicadas del dataframe
FR_df[FR_df.duplicated()].shape[0]

0

In [34]:
#Verificamos si los comentarios estan desabilitados para ciertos videos
FR_df.comments_disabled.value_counts()

comments_disabled
False    39835
True       889
Name: count, dtype: int64

In [35]:
#Verificamos si los likes estan  desabilitados para ciertos videos
FR_df.ratings_disabled.value_counts()

ratings_disabled
False    40020
True       704
Name: count, dtype: int64

In [36]:
#Veremos si los videos tuvieron un error o han sido removidos 
FR_df.video_error_or_removed.value_counts()

video_error_or_removed
False    40702
True        22
Name: count, dtype: int64